#### 一. 邻接矩阵
1. 什么是邻接矩阵  
  1. 邻接矩阵是图的一种表示方法, 能表示出顶点和边的关系  
  2. 邻接矩阵比邻接表在时间性能上更好, 因为大量查找变得操作可以通过矩阵直接定位, 而邻接表只能遍历顶点所在链表
1. 邻接矩阵 : 使用二维数组表示图中顶点和边的关系  
 1. 无向图 : $A[i][j]$=1表示两个定点之间存在边. 无向图的邻接矩阵是对称阵  
 2. 有向图 : $A[i][j]$=1表示两个定点之间存在从定点$i$ 到顶点$j$ 的边  
 3. 网络 : $A[i][j]$=n表示顶点之间边的权值
 
2. 邻接表编程实现的功能    
  1. 邻接表使用二维矩阵表示边集,一维向量表示顶点集
  3. 顶点上的操作 :  
    1. 获取顶点的一些信息  
    2. 插入新顶点 :  
     插入新顶点时, 顶点集增加, 边集中的入边和出边要设置成None  
    3. 删除顶点 :  
     删除顶点一切相关的边, 并修改相关顶点的入度出度
  4. 边的操作  
    1. 插入新边  
    2. 删除边  
     插入删除边都要修改对应的出度入度

In [125]:
###########################
###### 顶点和边的状态 #######
###########################
from enum import Enum
class Vstatus(Enum):
    '''顶点状态'''
    UNDISCOVERED = 1
    DISCOVERED = 2
    VISITED = 3
class EType(Enum):
    '''边的状态'''
    UNDETERMINED = 1
    TREE = 2
    CROSS = 3
    FORWARD = 4
    BACKWARD = 5

In [126]:
import sys

class Vertex(object):
    '''顶点'''
    def __init__(self,data,in_degree = 0,out_degree = 0,status = Vstatus.UNDISCOVERED,d_time = -1,
                f_time = -1,parent = -1,priority = sys.maxint):
        self.data = data
        self.in_degree = in_degree  # 入度
        self.out_degree = out_degree # 出度
        self.status = status
        self.d_time = d_time  # discover time
        self.f_time = f_time  # finish time
        self.parent = parent  # 顶点在遍历树中的父节点
        self.priority = priority # 顶点在遍历树中的优先级
        
class Edge(object):
    '''边'''
    def __init__(self,data,weight=-1,e_type=EType.UNDETERMINED):
        self.data = data
        self.weight = weight
        self.e_type = e_type
    
class Graph_Matrix(object):
    '''邻接矩阵表示的图'''
    def __init__(self):
        self.n = 0 # 顶点数
        self.e = 0 # 边数
        self.V = [] # 顶点集 : 一维数组
        self.E = [] # 边集 : 二维数组       
        
    ##### 顶点操作 
    def insert_vertex(self,vertex_data):
        '''插入顶点, 将其入边, 出边设成None'''
        self.n = self.n + 1
        # 出边设成None
        self.E.append([None for i in range(self.n)])
        # 入边设成None
        for i in range(self.n): 
            self.E[i].append(None)
        self.V.append(Vertex(vertex_data))
        
    def remove_vertes(self,i):
        '''删除顶点时,既要从顶点集中删除, 又要删除其关联变, 对应入度出度减1'''
        for j in range(self.n):
            self.E[j] = self.E[j][:i] + self.E[j][i+1:]  # 使用列表切片删除列表index下的元素
            self.V[j].in_degree = self.V[j].out_degree-1 # 出度减1
        for j in range(self.n):
            if self.exist_edge(i,j):
                self.V[j].in_degree = self.V[j].in_degree-1 # 入度减1
        self.E = self.E[:i]+self.E[i+1:]  # 删除该顶点所有出边集合
        v_remove = self.V[i]
        self.V = self.V[:i]+self.V[i+1:]  
        return v_remove
    
    ### 边操作
    def exist_edge(self,i,j):
        return self.E[i][j] is not None
    def insert_edge(self,edge_data,w,i,j):
        if self.exist_edge(i,j):  # 确保边尚不存在
            print 'return'
            return 
        self.E[i][j] = Edge(edge_data,w)
        self.e = self.e+1 
        self.V[i].out_degree = self.V[i].out_degree+1
        self.V[j].in_degree = self.V[j].in_degree+1
    def remove_edge(self,i,j):
        '''删除边'''
        e_bak = self.E[i][j]
        self.E[i][j] = None
        e = e-1
        self.V[i].out_degree = self.V[i].out_degree-1
        self.V[j].in_degree = self.V[j].in_degree-1
        return e_bak

#### 二. 遍历
1. 深度优先    
  1. 深度优先遍历, 优先选取最后一个被访问到的顶点的邻居  
  2. 以顶点$s$为基点的$DFS$搜索, 首先访问$s$, 再从所有尚未访问到的邻居中任取其一, 并以之为基点, 再次递归进行$DFS$搜索  
   因此, 顶点被访问到的次序, 类似于树的先序遍历; 而各顶点被访问完毕的次序, 类似后序遍历  
  3. 如下程序展示了深度优先遍历如下图  
  <img src='img/shenduyouxian.png' height='30%' width='30%'>
2. 广度优先  
  1. 广度优先遍历, 越早被访问到的顶点, 其邻居优先被访问  
  2. 广度优先遍历在发现顶点后立刻访问, 所以顶点的d_time(发现时间)和f_time(完成时间)相等.  
   而深度优先遍历不同, 深度优先在发现顶点后, 不能立即访问, 先把自己和其邻居入栈, 等待最后回溯到该顶点时, 才能弹栈访问

In [127]:
class GraphTraverse(object):
    @staticmethod
    def DFS(start,graph,visit):
        '''start: 从哪个顶点开始遍历, 因为存在多连通分量的情况, 所以需要遍历整个节点看师傅都被访问到了
           graph: 需要被遍历的图
           visit: 访问函数
                `visit(current_idx,current_vertex)`: 2个参数, 顶点index和顶点对象
        '''
        edge_matrix = graph.E
        vertexs = graph.V
        n = graph.n  # 顶点个数
        
        clock = 1
        stack = []
        
        traverse_idx_order = range(start,n)+range(0,start) # 从开始位置向后遍历
        
        for start in traverse_idx_order:
            if vertexs[start].status == Vstatus.UNDISCOVERED:
                stack.append(start)
                vertexs[start].d_time = clock
                clock = clock + 1
            while len(stack) >0:
                s = stack[-1]
                # 所有邻居节点
                neighbors_idxs = [i for i in range(n) if edge_matrix[s][i] is not None]
                undiscover_neighbors = [i for i in neighbors_idxs if vertexs[i].status==Vstatus.UNDISCOVERED]
                disovered_neighbors = [i for i in neighbors_idxs if vertexs[i].status==Vstatus.DISCOVERED]
                visited_neighbors = [i for i in neighbors_idxs if vertexs[i].status==Vstatus.VISITED]

                # 1.如果栈顶顶点没有'未访问'的邻居, 则可弹栈进行访问
                if len(undiscover_neighbors)==0:
                    current_idx = stack.pop() # 弹栈
                    current_vertex = vertexs[current_idx]
                    current_vertex.status = Vstatus.VISITED
                    current_vertex.f_time = clock
                    visit(current_idx,current_vertex)
                # 2.未发现的邻居节点
                else:
                    for i in undiscover_neighbors:
                        stack.append(i) # 未发现的邻居节点先入栈,但不访问.
                        vertexs[i].status = Vstatus.DISCOVERED
                        vertexs[i].parent = s
                        vertexs[i].d_time = clock
                        edge_matrix[s][i].status = EType.TREE

                # 3.已发现的邻居节点
                for i in disovered_neighbors:
                    edge_matrix[s][i].status = EType.BACKWARD

                # 4.已访问的邻居节点
                for i in visited_neighbors:
                    edge_matrix[s][i].status = (EType.CROSS if vertexs[s].d_time < vertexs[i].d_time else EType.FORWARD)

                clock = clock + 1

    @staticmethod       
    def BFS(start,graph):
        '''广度优先遍历'''
        return
        

In [128]:
if __name__ == '__main__':
    graph = Graph_Matrix()
    graph.insert_vertex('A')
    graph.insert_vertex('B')
    graph.insert_vertex('C')
    graph.insert_vertex('D')
    graph.insert_vertex('E')
    graph.insert_vertex('F')
    graph.insert_vertex('G')
    
    graph.insert_edge(-1,-1,0,1)
    graph.insert_edge(-1,-1,0,2)
    graph.insert_edge(-1,-1,0,5)
    graph.insert_edge(-1,-1,3,0)
    graph.insert_edge(-1,-1,3,4)
    graph.insert_edge(-1,-1,4,5)
    graph.insert_edge(-1,-1,5,6)
    graph.insert_edge(-1,-1,1,2)
    graph.insert_edge(-1,-1,6,2)
    
    def visit(index,vertex):
        print 'index->vertex.data: %s->%s, dtime:%s, ftime:%s'%(index,vertex.data,vertex.d_time,vertex.f_time)
    GraphTraverse.DFS(3,graph,visit)

index->vertex.data: 2->C, dtime:5, ftime:6
index->vertex.data: 6->G, dtime:4, ftime:7
index->vertex.data: 5->F, dtime:3, ftime:8
index->vertex.data: 4->E, dtime:2, ftime:9
index->vertex.data: 1->B, dtime:10, ftime:11
index->vertex.data: 0->A, dtime:2, ftime:12
index->vertex.data: 3->D, dtime:1, ftime:13


#### 三. 拓扑排序  
1. 何为拓扑排序  
 给定一个实际应用, 依赖时间发生顺序形成有向图, 将该有向图在'相容'的条件下, 转换成一个线性序列. 该线性序列称为图的'拓扑排序'  
 '相容'指每一顶点, 都不会通过边, 指向此序列中的前驱节点  
2. 任一不含回路的有向无环图, 都尤其对应的拓扑排序
 有向无环图的拓扑排序并不唯一  
3. 一种递归实现的拓扑排序  
 因为有向无环图都至少含有1个入度为0的点.(否则的话, 图存在回路). 因此, 将该入度为0的点和从其出发的边从图中删去后, 剩下的图形仍然是有向无环图, 依然存在拓扑序列. 因此, 从递归的角度看:  
   1. 递归基 : 只剩一个顶点时, 拓扑序列为该顶点  
   2. 递归条件 : 每次从图中删除入度为0的顶点, 剩下的图队规执行进行拓扑排序
4. 使用深度优先遍历进行拓扑排序 
  1. 上面拓扑排序中, 不断删除入度为0的点进行递归. 现在, 假如我们每次从图中删除出度为0的顶点, 则这个删除的顺序构成拓扑排序的逆序
  1. 深度优先遍历, 在顶点没有UNDISCOVERED状态的邻居时可以访问该顶点, 恰恰能满足了删除出度为0的顶点构成拓扑逆序的要求. 只是从入度为0的顶点开始深度优先遍历, 从而得到拓扑排序的逆序  
    1. python使用list[::-1]反转list

In [174]:
import numpy as np
class TopologicalSort(object):
    @staticmethod
    def t_sort(graphy,visit):
        n = graphy.n
        zero_out = [None for i in range(n)]
        # 查找第一个出度为0的点的index. 翻转edge矩阵在查找, 相当于查找原来edge矩阵哪一个为全None
        start = np.array(graph.E).T.tolist().index(zero_out)  
        GraphTraverse.DFS(start,graph,visit)

In [175]:
if __name__ == '__main__':
    graph = Graph_Matrix()
    graph.insert_vertex('A')
    graph.insert_vertex('B')
    graph.insert_vertex('C')
    graph.insert_vertex('D')
    graph.insert_vertex('E')
    graph.insert_vertex('F')
    
    graph.insert_edge(-1,-1,0,2)
    graph.insert_edge(-1,-1,0,3)
    graph.insert_edge(-1,-1,1,2)
    graph.insert_edge(-1,-1,2,3)
    graph.insert_edge(-1,-1,2,4)
    graph.insert_edge(-1,-1,2,5)
    graph.insert_edge(-1,-1,4,5)
    
    stack = []
    def visit(index,vertex):
        stack.append(vertex.data)
    TopologicalSort.t_sort(graph,visit)
    print stack[::-1]

['B', 'A', 'C', 'E', 'F', 'D']


In [161]:
arr = np.arange(15).reshape(3,5)
arr.T.to

array([[ 0,  5, 10],
       [ 1,  6, 11],
       [ 2,  7, 12],
       [ 3,  8, 13],
       [ 4,  9, 14]])

#### 四. 双联通分量分解
略

#### 五.优先级搜索
略

#### 六. 最小支撑树
1. 

#### 七. 最短路径

In [180]:
import sys
class Dijkstra(object):
    @staticmethod
    def route(graph,src_idx):
        n = graph.n
        vertexs = graph.V
        edge_matrix = graph.E
        # 节点的weight属性, 记录源点到目标点路径的权值总和
        vertexs[src_idx].weight = 0 
        # 源点的邻居节点
        neighbors_idxs = [i for i in range(n) if edge_matrix[src_idx][i] is not None]
        # 初始化权值向量, 每个元素为该点到源点的距离, 若该店到源点之间没有边, 则将距离设为int最大值
        initweights = [edge_matrix[src_idx][i] if i in neighbors_idxs else sys.maxint for i in range(n)]

        for i in range(n):
            vertexs[i].parent = src_idx
            
            

SyntaxError: invalid syntax (<ipython-input-180-630aaab9ef4d>, line 9)

In [179]:
[1 if i>2 else -1 for i in range(4)]

[-1, -1, -1, 1]